# Combining pyktok and the reasearch api

In [1]:
import requests
import json

import pyktok as pyk
pyk.specify_browser('firefox')

from IPython.display import Video

We strongly recommend you run 'specify_browser' first, which will allow you to run pyktok's functions without using the browser_name parameter every time. 'specify_browser' takes as its sole argument a string representing a browser installed on your system, e.g. "chrome," "firefox," "edge," etc.


Setup secrets and get access token.

In [2]:
# setup api secrets
with open('../secrets/tiktok.json') as f:
    secrets = json.load(f)
    
client_key = secrets['client_key']
client_secret = secrets['client_secret']

# get access token
url = "https://open.tiktokapis.com/v2/oauth/token/"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Cache-Control": "no-cache"
}
data = {
    "client_key": client_key,
    "client_secret": client_secret,
    "grant_type": "client_credentials"
}
response = requests.post(url, headers=headers, data=data)
response_json = response.json()
access_token = response_json['access_token']

Request videos from TikTok's research API.

In [3]:
fields = 'id,video_description,create_time,region_code,share_count,view_count,like_count,comment_count,music_id,hashtag_names,username,effect_ids,playlist_id,voice_to_text'
url = f'https://open.tiktokapis.com/v2/research/video/query/?fields={fields}'
headers = {
    "authorization": f"bearer {access_token}",
    "Content-Type": "application/json"
}
data = { 
    "query": {
        "and": [
            { "operation": "EQ", "field_name": "region_code", "field_values": ["DK"] },
            { "operation": "EQ", "field_name": "hashtag_name", "field_values": ["viral"] }
        ]
    }, 
    "start_date":   "20240101",
    "end_date":     "20240101",
    "max_count": 20
}
response = requests.post(url, headers=headers, json=data)
response.status_code

500

Extract the videos

In [4]:
videos = response.json()['data']['videos']
videos

KeyError: 'data'

Download videos with pyktok

In [ ]:
for video in videos:
    vid = video['id']
    username = video['username']
    vurl = f'https://www.tiktok.com/@{username}/video/{vid}'
    print(vurl)
    
    try:
        pyk.save_tiktok(
            vurl,
            True,
            'video_data.csv',
            'firefox'
        )
    except Exception as e:
        print(e)
        continue

https://www.tiktok.com/@zophie1984/video/7319283514413813025
Saved video
 https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-ve-0068-euttp/oICQPNI3EIEJWPWYN0ZiBTBTazijwBZAL8UAQ/?a=1988&bti=ODszNWYuMDE6&ch=0&cr=3&dr=0&lr=tiktok_m&cd=0%7C0%7C1%7C&cv=1&br=2194&bt=1097&cs=0&ds=3&ft=4fUEKMFw8Zmo0xs5J-4jV5B1upWrKsd.&mime_type=video_mp4&qs=0&rc=O2llOGk6Zzc3ODg5ZDY6NEBpM3BycnE5cmVvcDMzZjczM0BgMy4vLmBeXl8xXy02YjU1YSNgcm4zMmQ0LTZgLS1kMWNzcw%3D%3D&btag=e00088000&expire=1717088052&l=20240530105357F5EC4E147238412A0C4B&ply_type=2&policy=2&signature=096ba759bf46187042d4ba193c8f156a&tk=tt_chain_token 
to
 /home/mahf/Desktop/thesis/notebooks
Saved metadata for video
 https://www.tiktok.com/@zophie1984/video/7319283514413813025 
to
 /home/mahf/Desktop/thesis/notebooks
https://www.tiktok.com/@ruwey_s/video/7319283330673839392
Invalid URL '': No scheme supplied. Perhaps you meant https://?
https://www.tiktok.com/@vesselcompanyco/video/7319283061122698529
Saved video
 https://v16-webapp-pr

In [ ]:
vid = videos[0]['id']
username = videos[0]['username']
Video(f'@{username}_video_{vid}.mp4', embed=True)